# Recomend Artist Based on Embeddings

In [19]:
%run ./wikipedia_setup.ipynb

In [20]:
import gensim
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.neighbors import KDTree
from sklearn.cluster import AgglomerativeClustering

In [3]:
pd.set_option('display.max_rows', 100)

## Configure DB & Load Model

In [4]:
db = sqlite3.Connection("var/data/music.db")

In [5]:
doc_model = Doc2Vec.load("var/models/artist_detail_model.d2v")

2021-11-02 08:54:45,284 - gensim.utils - INFO - loading Doc2Vec object from var/models/artist_detail_model.d2v
2021-11-02 08:54:45,543 - gensim.utils - INFO - loading dv recursively from var/models/artist_detail_model.d2v.dv.* with mmap=None
2021-11-02 08:54:45,545 - gensim.utils - INFO - loading wv recursively from var/models/artist_detail_model.d2v.wv.* with mmap=None
2021-11-02 08:54:45,546 - gensim.utils - INFO - setting ignored attribute cum_table to None
2021-11-02 08:54:48,988 - gensim.utils - INFO - Doc2Vec lifecycle event {'fname': 'var/models/artist_detail_model.d2v', 'datetime': '2021-11-02T08:54:48.988341', 'gensim': '4.1.2', 'python': '3.7.6 (tags/v3.7.6:43364a7ae0, Dec 19 2019, 00:42:30) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


## Load Artst + Embeddings

In [6]:
artists = pd.read_sql(f"""
    select *
    from artist_embedding
""", db)
artists['DetailEmbedding'] = artists.DetailEmbedding.apply(lambda de: np.array(list(map(float, de.split(",")))))

## Find Neighbors For Artists

Use a KDTree to find artist close to others.

In [7]:
# Create a KDTree over embeddings to search
ad_emb = np.vstack(artists.DetailEmbedding)
ad_neighbors = KDTree(ad_emb)

In [8]:
def get_similar_artists(artist_pattern, n):
    s_artist = artists[artists.Artist == artist_pattern].iloc[0]
    n_distances, neighbors = ad_neighbors.query(s_artist.DetailEmbedding.reshape(1, -1), k=n)

    neighbor_rows = artists.iloc[neighbors[0]].copy()
    neighbor_rows['Distance'] = n_distances[0]
    return neighbor_rows

In [16]:
test_artists = [
    "Electric Light Orchestra",
    "Dr. Dre",
    "Nirvana",
    "Nine Inch Nails",
    "The Doors",
    "Tyler, the Creator",
    "King Gizzard & the Lizard Wizard",
    "Snarky Puppy",
    "Pink Floyd",
    "Metallica",
    "Animals as Leaders",
    "Johnny Cash",
    "Thelonious Monk",
    "Frank Zappa",
    "Crystal Castles",
    "Aesop Rock",
    "Wu-Tang Clan"
]

In [18]:
for ta in test_artists:
    display(HTML(f"<h2>{ta}</h2>"))
    display(get_similar_artists(ta, 25))

,Artist,ArtistLink,DetailEmbedding,Distance
3007,Electric Light Orchestra,https://en.wikipedia.org/wiki/Electric_Light_O...,"[-0.387232, -1.6674888, -1.0893855, 0.4903128,...",0.000000
4628,Jeff Lynne's ELO,https://en.wikipedia.org/wiki/Electric_Light_O...,"[-0.43554413, -1.4091171, -1.2342099, 0.703724...",1.011800
4627,Jeff Lynne,https://en.wikipedia.org/wiki/Jeff_Lynne,"[-0.6362788, -0.35304442, -1.5312362, -1.50112...",7.605310
9770,The Move,https://en.wikipedia.org/wiki/The_Move,"[-2.0066159, -0.9716653, -1.9260595, 0.3975553...",7.742207
8042,Roy Wood,https://en.wikipedia.org/wiki/Roy_Wood,"[-1.406831, -0.52842104, -1.3930508, -0.056852...",8.342801
7180,Page and Plant,https://en.wikipedia.org/wiki/Page_and_Plant,"[-0.026165288, -1.8567859, -0.80827177, 1.6971...",9.401899
916,"Beck, Bogert & Appice","https://en.wikipedia.org/wiki/Beck,_Bogert_%26...","[0.69869906, -1.2511948, -1.5157948, 1.6605303...",9.426510
9655,The Jeff Beck Group,https://en.wikipedia.org/wiki/The_Jeff_Beck_Group,"[-0.068981744, -2.5164418, -0.70135754, 0.3847...",9.443006
9934,The Shadows,https://en.wikipedia.org/wiki/The_Shadows,"[-1.3071729, 0.010029454, -1.4905453, -0.48784...",9.876729
6899,Nigel Olsson,https://en.wikipedia.org/wiki/Nigel_Olsson,"[-0.83315456, -1.9111655, -1.8051133, -0.04000...",10.160539


,Artist,ArtistLink,DetailEmbedding,Distance
2817,Dr. Dre,https://en.wikipedia.org/wiki/Dr._Dre,"[1.2855527, -0.0500333, -1.283109, -3.730988, ...",0.000000
6721,N.W.A,https://en.wikipedia.org/wiki/N.W.A,"[0.9409786, -1.3837426, -2.867902, -1.1560463,...",8.350523
3095,Eminem,https://en.wikipedia.org/wiki/Eminem,"[-0.5083133, -2.3587565, -2.728328, -2.3337731...",8.480308
6034,Makaveli,https://en.wikipedia.org/wiki/Tupac_Shakur,"[1.4039732, -1.9706135, -1.3985945, -1.7411282...",8.913099
45,2Pac,https://en.wikipedia.org/wiki/Tupac_Shakur,"[1.367418, -1.822259, -1.4461393, -1.6682366, ...",8.965055
9797,The Notorious B.I.G.,https://en.wikipedia.org/wiki/The_Notorious_B....,"[1.0675318, 0.08797085, -0.092092246, -2.70204...",8.974415
65,50 Cent,https://en.wikipedia.org/wiki/50_Cent,"[0.34396002, -0.42005056, -2.005829, -2.337675...",9.007605
2938,Eazy-E,https://en.wikipedia.org/wiki/Eazy-E,"[1.8762194, 0.7717415, -3.0955117, -0.19510518...",9.010661
6982,Notorious B.I.G.,https://en.wikipedia.org/wiki/Notorious_B.I.G.,"[1.0067303, 0.14022663, -0.1228546, -2.7223122...",9.014412
9442,The D.O.C.,https://en.wikipedia.org/wiki/The_D.O.C.,"[0.89909476, 0.50968736, -3.1806424, -0.389132...",9.015563


,Artist,ArtistLink,DetailEmbedding,Distance
6936,Nirvana,https://en.wikipedia.org/wiki/Nirvana_(band),"[0.28319836, 1.0070648, -1.67811, -1.3276137, ...",0.000000
5425,Kurt Cobain,https://en.wikipedia.org/wiki/Kurt_Cobain,"[-0.6089347, 0.43105546, 0.28856748, -0.839133...",8.385896
9015,Sweet 75,https://en.wikipedia.org/wiki/Sweet_75,"[-0.7353841, 1.6106955, -2.2669494, 0.27626446...",8.970280
1970,Ciccone Youth,https://en.wikipedia.org/wiki/Ciccone_Youth,"[0.65132445, 0.0708135, 0.13343593, 0.49504712...",9.272674
8669,Sonic Youth,https://en.wikipedia.org/wiki/Sonic_Youth,"[0.64971334, -0.0028599463, 0.1993272, 0.41892...",9.420928
4549,Jane's Addiction,https://en.wikipedia.org/wiki/Jane%27s_Addiction,"[0.2790259, -0.31124517, -0.7799071, 0.9336407...",9.713855
3414,Foo Fighters,https://en.wikipedia.org/wiki/Foo_Fighters,"[-0.9212452, -0.05392002, -1.9442375, -0.02863...",10.033896
2553,Dee Gees,https://en.wikipedia.org/wiki/Foo_Fighters,"[-1.1625684, 0.15792744, -2.187276, 0.2859022,...",10.043255
10647,Violent Femmes,https://en.wikipedia.org/wiki/Violent_Femmes,"[-0.8325488, 0.5266511, -0.4793426, 0.6328272,...",10.084339
4244,Hüsker Dü,https://en.wikipedia.org/wiki/H%C3%BCsker_D%C3%BC,"[-0.12512562, -0.945129, -1.4524004, 0.6408224...",10.131278


,Artist,ArtistLink,DetailEmbedding,Distance
6928,Nine Inch Nails,https://en.wikipedia.org/wiki/Nine_Inch_Nails,"[1.060734, -0.25217286, -1.725567, -0.23710454...",0.000000
6929,"Nine Inch Nails, Jane's Addiction and Street S...",https://en.wikipedia.org/wiki/Nine_Inch_Nails,"[1.1367074, -0.12394637, -1.5993673, -0.210905...",0.909239
10322,Tool,https://en.wikipedia.org/wiki/Tool_(band),"[2.425879, 0.058206216, -2.363124, 0.30551565,...",8.759670
8531,Skinny Puppy,https://en.wikipedia.org/wiki/Skinny_Puppy,"[0.34352, 1.5137974, -1.1172683, -0.07200697, ...",9.132811
6116,Marilyn Manson,https://en.wikipedia.org/wiki/Marilyn_Manson_(...,"[-0.73967016, 0.0073882705, -1.2480562, -0.800...",10.029504
6487,Ministry,https://en.wikipedia.org/wiki/Ministry_(band),"[1.0117749, -0.5220631, -1.9405884, 0.31129205...",10.683318
7585,Prototype,https://en.wikipedia.org/wiki/Prototype_(band),"[0.16111772, -0.26046318, -1.1244873, 2.593899...",10.848428
7693,Rage Against the Machine,https://en.wikipedia.org/wiki/Rage_Against_the...,"[-0.862439, -2.2870255, -0.6182816, 1.4990528,...",10.878702
9953,The Smashing Pumpkins,https://en.wikipedia.org/wiki/The_Smashing_Pum...,"[3.0714421, -1.4383739, -2.0018141, -0.4146463...",10.880068
4203,How to Destroy Angels,https://en.wikipedia.org/wiki/How_to_Destroy_A...,"[0.14115435, 0.21004239, -0.5088485, 0.1839790...",10.909624


,Artist,ArtistLink,DetailEmbedding,Distance
9489,The Doors,https://en.wikipedia.org/wiki/The_Doors,"[1.2109855, -1.7594427, -0.42090923, -1.716139...",0.000000
10712,Warren Zevon,https://en.wikipedia.org/wiki/Warren_Zevon,"[0.51454633, 0.5419159, -1.0498676, -1.7367202...",9.561460
10206,Tim Hardin,https://en.wikipedia.org/wiki/Tim_Hardin,"[0.92015576, -1.6389208, -1.1367968, -0.104717...",10.045448
2924,Eagles,https://en.wikipedia.org/wiki/Eagles_(band),"[-0.16062242, 0.7036986, -1.9971437, -0.999429...",10.173196
9874,The Ramones,https://en.wikipedia.org/wiki/The_Ramones,"[0.3320279, 0.053347073, -1.1003983, 1.2281626...",10.305949
5849,Lou Reed,https://en.wikipedia.org/wiki/Lou_Reed,"[0.35972795, 0.43413144, 1.1458242, -1.1807466...",10.340633
7712,Ramones,https://en.wikipedia.org/wiki/Ramones,"[0.43494982, 0.1777931, -1.073549, 1.2050525, ...",10.410459
10275,Tom Petty and the Heartbreakers,https://en.wikipedia.org/wiki/Tom_Petty_and_th...,"[0.5346684, 1.3130541, -2.0048978, -1.4241321,...",10.453282
10058,"The Velvet Underground, Nico",https://en.wikipedia.org/wiki/The_Velvet_Under...,"[0.30003092, 0.52220464, 2.135478, -0.05985224...",10.469569
2772,Don Felder,https://en.wikipedia.org/wiki/Don_Felder,"[0.40788862, 1.1514858, -0.8904536, -1.0150033...",10.471077


...

,Artist,ArtistLink,DetailEmbedding,Distance
10131,Thelonious Monk,https://en.wikipedia.org/wiki/Thelonious_Monk,"[1.3236248, -1.4610785, 2.1854656, 0.8591331, ...",0.000000
1529,Bud Powell,https://en.wikipedia.org/wiki/Bud_Powell,"[-0.12226143, -1.0133735, 2.7336912, 0.3095159...",6.868418
1815,Charles Mingus,https://en.wikipedia.org/wiki/Charles_Mingus,"[0.4318651, -0.20629476, 1.2453475, 1.299351, ...",7.600607
1822,Charlie Parker,https://en.wikipedia.org/wiki/Charlie_Parker,"[0.90149647, -1.4503442, 2.5635028, 0.65616006...",7.768442
6467,Miles Davis,https://en.wikipedia.org/wiki/Miles_Davis,"[1.8203194, -1.3783013, 1.4133562, -0.08504401...",7.884450
6036,Mal Waldron,https://en.wikipedia.org/wiki/Mal_Waldron,"[0.09390967, -0.32788667, 2.3756695, 1.045799,...",8.229007
8673,Sonny Rollins,https://en.wikipedia.org/wiki/Sonny_Rollins,"[0.48749408, -1.9948779, 0.63102, 1.153534, -3...",8.235637
3051,Elmo Hope,https://en.wikipedia.org/wiki/Elmo_Hope,"[1.1894351, 0.117529616, 0.7056927, 1.8157315,...",8.317331
4424,J. J. Johnson,https://en.wikipedia.org/wiki/J._J._Johnson,"[1.0776122, -1.0214149, 0.26703256, -0.1398547...",8.491367
977,Benny Goodman,https://en.wikipedia.org/wiki/Benny_Goodman,"[0.749154, -2.868556, 0.46563745, -0.08983608,...",8.529526


,Artist,ArtistLink,DetailEmbedding,Distance
3466,Frank Zappa,https://en.wikipedia.org/wiki/Frank_Zappa,"[1.3201275, -1.6736283, 0.49244577, 1.7891124,...",0.000000
3468,Frank Zappa and The Mothers,https://en.wikipedia.org/wiki/Frank_Zappa,"[1.2510762, -1.4596589, 0.5700625, 1.8608723, ...",0.742593
3469,Frank Zappa and The Mothers of Invention,https://en.wikipedia.org/wiki/Frank_Zappa,"[1.233183, -1.4448514, 0.64113045, 1.8178601, ...",0.870435
3467,Frank Zappa & The Mothers of Invention with Ca...,https://en.wikipedia.org/wiki/Frank_Zappa,"[1.2147509, -1.4146931, 0.5676594, 1.9262733, ...",0.877467
9768,The Mothers of Invention,https://en.wikipedia.org/wiki/The_Mothers_of_I...,"[1.1942087, -0.80340695, 0.6805217, 1.8576227,...",8.336503
1681,Captain Beefheart,https://en.wikipedia.org/wiki/Captain_Beefheart,"[3.2096143, -1.3189019, 3.1504781, -0.00977150...",11.386404
1682,Captain Beefheart & his Magic Band,https://en.wikipedia.org/wiki/Captain_Beefheart,"[3.0575578, -1.3231854, 3.06541, -0.04069842, ...",11.397961
1683,Captain Beefheart & the Magic Band,https://en.wikipedia.org/wiki/Captain_Beefheart,"[2.9553223, -1.1626133, 3.130786, 0.15107195, ...",11.512735
6446,Mike Bloomfield/Al Kooper/Stephen Stills,https://en.wikipedia.org/wiki/Mike_Bloomfield,"[3.100735, -0.67464507, 0.01797964, 0.2652832,...",12.477339
6445,Mike Bloomfield & Al Kooper,https://en.wikipedia.org/wiki/Mike_Bloomfield,"[3.3074453, -0.54840726, 0.083469346, 0.289399...",12.595344


,Artist,ArtistLink,DetailEmbedding,Distance
2206,Crystal Castles,https://en.wikipedia.org/wiki/Crystal_Castles,"[-2.3786213, 1.720056, -0.6421294, 0.97738504,...",0.000000
347,Alice Glass,https://en.wikipedia.org/wiki/Alice_Glass,"[-1.6071162, 2.9547224, -0.7794519, -0.3689876...",6.572272
9466,The Departure,https://en.wikipedia.org/wiki/The_Departure,"[-0.79209393, 0.72695607, -0.7511203, 0.550726...",8.496254
9918,The Safety Fire,https://en.wikipedia.org/wiki/The_Safety_Fire,"[-2.9172258, 0.049108624, -1.7587222, 3.059865...",8.608218
1144,Black Peaks,https://en.wikipedia.org/wiki/Black_Peaks_(band),"[-0.24953727, -0.7948105, -1.5882419, 0.258613...",8.638066
10363,Trailer Trash Tracys,https://en.wikipedia.org/wiki/Trailer_Trash_Tr...,"[-1.5663157, -0.12509324, -0.90989083, 0.38639...",8.678374
1178,Blanck Mass,https://en.wikipedia.org/wiki/Blanck_Mass,"[-0.7644815, -0.15446673, 0.27153748, 0.812709...",8.679305
1188,Bleached,https://en.wikipedia.org/wiki/Bleached,"[-1.1339824, 0.21663594, -1.3985174, -0.196395...",8.725605
10783,White Rose Movement,https://en.wikipedia.org/wiki/White_Rose_Movement,"[-1.6042454, -0.73409456, -1.1034228, 0.423407...",8.811384
1966,Chvrches,https://en.wikipedia.org/wiki/Chvrches,"[-2.0698385, 0.9466864, 1.8596033, -0.27650324...",8.869539


,Artist,ArtistLink,DetailEmbedding,Distance
217,Aesop Rock,https://en.wikipedia.org/wiki/Aesop_Rock,"[1.5709599, -2.3995297, -0.96275586, 1.1011693...",0.000000
218,Aesop Rock and Blockhead,https://en.wikipedia.org/wiki/Aesop_Rock,"[1.5408995, -2.2402227, -0.89001673, 1.0210788...",1.093930
2999,El-P,https://en.wikipedia.org/wiki/El-P,"[1.6139878, -1.9222982, -1.3806736, 0.9058491,...",8.592907
1206,Blockhead,https://en.wikipedia.org/wiki/Blockhead_(music...,"[0.17684881, -0.9057878, 0.0033948997, -0.8360...",9.582348
4960,Jonwayne,https://en.wikipedia.org/wiki/Jonwayne,"[1.32209, -0.45423606, 0.4005551, -0.27543113,...",9.650269
7093,Open Mike Eagle,https://en.wikipedia.org/wiki/Open_Mike_Eagle,"[-0.2557377, -0.70402896, 1.1051924, -0.410294...",9.755743
4158,Homeboy Sandman,https://en.wikipedia.org/wiki/Homeboy_Sandman,"[1.5858854, -0.054282162, -0.8565406, -0.73803...",9.844695
7659,R.A. the Rugged Man,https://en.wikipedia.org/wiki/R.A._the_Rugged_Man,"[0.46239978, 0.12132466, -0.3845071, 1.0211447...",10.024866
8698,Souls of Mischief,https://en.wikipedia.org/wiki/Souls_of_Mischief,"[0.2850436, -2.064198, -1.1176888, 0.7820858, ...",10.050105
7167,P.O.S,https://en.wikipedia.org/wiki/P.O.S_(rapper),"[1.0501184, 1.230038, -0.6957069, 1.671424, 0....",10.057933


,Artist,ArtistLink,DetailEmbedding,Distance
10897,Wu-Tang Clan,https://en.wikipedia.org/wiki/Wu-Tang_Clan,"[3.353331, -2.6517022, -2.7383525, -0.23463415...",0.000000
3700,Ghostface Killah,https://en.wikipedia.org/wiki/Ghostface_Killah,"[2.2753766, -1.8000417, -2.5112667, -0.6220655...",6.205513
7690,Raekwon,https://en.wikipedia.org/wiki/Raekwon,"[3.3098454, -4.131073, -1.6355907, 0.632834, -...",6.706944
4370,Inspectah Deck,https://en.wikipedia.org/wiki/Inspectah_Deck,"[3.6703475, -3.0340774, -1.6248453, 1.3381405,...",7.200901
7669,RZA,https://en.wikipedia.org/wiki/RZA,"[3.2637715, -2.1416008, -1.5330702, -0.5621785...",7.207109
6360,Method Man,https://en.wikipedia.org/wiki/Method_Man,"[2.290064, -2.0957198, -1.3429325, -0.09811122...",7.263314
7670,RZA as Bobby Digital,https://en.wikipedia.org/wiki/RZA,"[3.0629053, -2.1711862, -1.4218601, -0.4218399...",7.374556
6211,Masta Killa,https://en.wikipedia.org/wiki/Masta_Killa,"[1.6964147, -2.2345314, -1.9673363, 0.9060108,...",7.655183
1679,Cappadonna,https://en.wikipedia.org/wiki/Cappadonna,"[2.4734163, -1.0585515, -2.5639353, 0.36960834...",7.683990
6226,Mathematics,https://en.wikipedia.org/wiki/Mathematics_(pro...,"[2.1434464, -2.6100993, -1.9069554, -0.1222976...",7.915114


## Try Mixing Artists To Find References

In [131]:
def find_close_mixed(artist_names, n=20):
    
    ref_artists = artists[artists.Artist.isin(artist_names)]
    ref_emb = np.vstack(ref_artists.DetailEmbedding).mean(axis=0)
    
    scores, ids = ad_neighbors.query(ref_emb.reshape(1, -1), k=n)
    close_artists = artists.iloc[ids[0]].copy()
    close_artists['SimScore'] = scores[0]
    close_artists = close_artists.sort_values(by='SimScore', ascending=True)
    
    return close_artists

In [140]:
find_close_mixed(['The Cure', 'Animals as Leaders', 'System of a Down'])

,Artist,ArtistLink,DetailEmbedding,Cluster,SimScore
629,Arkaea,https://en.wikipedia.org/wiki/Arkaea,"[-0.6553504, 0.7150783, -1.8626635, 1.8232595,...",80,6.364110
4339,In Case of Fire,https://en.wikipedia.org/wiki/In_Case_of_Fire,"[-2.0683682, -0.13448018, -1.4897661, 0.505864...",26,6.479470
10753,Wellwater Conspiracy,https://en.wikipedia.org/wiki/Wellwater_Conspi...,"[-0.11677661, -0.7548648, -1.8598219, 1.609214...",46,6.548231
754,AxeWound,https://en.wikipedia.org/wiki/AxeWound,"[-1.7212716, -0.46619728, -1.6760972, 1.593793...",46,6.574576
6529,Mob Rules,https://en.wikipedia.org/wiki/Mob_Rules_(band),"[-1.0012428, -1.2749624, -1.7759936, 1.1756712...",46,6.839533
3328,Fight,https://en.wikipedia.org/wiki/Fight_(band),"[-0.97717875, -0.4489926, -1.7863394, 2.228654...",46,6.873934
7081,One Minute Silence,https://en.wikipedia.org/wiki/One_Minute_Silence,"[-0.53409046, -0.51835215, 0.45480764, 2.85845...",26,6.899177
9777,The Mute Gods,https://en.wikipedia.org/wiki/The_Mute_Gods,"[-0.5455548, -0.12845905, -0.7078378, 1.512511...",46,6.954371
6846,New Device,https://en.wikipedia.org/wiki/New_Device,"[-1.2462641, -0.26549223, -1.3422089, 1.258723...",46,6.960790
8716,Spawn of Possession,https://en.wikipedia.org/wiki/Spawn_of_Possession,"[-1.0086046, 0.27147156, -1.0598925, 3.3937385...",46,7.072101


## Cluster Artists

In [105]:
cm = AgglomerativeClustering(distance_threshold=50, n_clusters=None)

In [106]:
artists['Cluster'] = cm.fit_predict(ad_emb)

In [107]:
artist_clusters = (artists
    .groupby('Cluster')
    .size()
    .reset_index()
    .rename(columns={0: 'Count'})
    .sort_values(by='Count', ascending = False)
)

In [142]:
artist_clusters = pd.merge(
    artist_clusters,
    (artists
        .groupby('Cluster')
        .apply(lambda c_df: list(c_df.sample(10).Artist if len(c_df) > 10 else c_df.Artist))
        .reset_index()
        .rename(columns={0: 'SampleArtists'})
    ),
    on = 'Cluster'
)

In [148]:
albums = pd.read_sql("select Artist, Album, Genre, Year from wp_albums_year_lists", db)

In [157]:
def top_genres(df):
    return (df[df.Genre.notnull() & (df.Genre != '')]
        .groupby('Genre')
        .size()
        .reset_index()
        .rename(columns={0: 'Count'})
        .sort_values(by='Count', ascending=False)
        .head(5)
    )

top_genres(albums)

,Genre,Count
1627,Hip hop,527
1861,Indie rock,346
727,Country,311
2401,Pop,278
3068,Rock,266


In [161]:
album_artist_cluster = pd.merge(
    albums,
    artists[['Artist', 'Cluster']],
    on = 'Artist'
)

def get_album_cluster_info(df):
    
    t_genres = list(top_genres(df).Genre.values)
    
    return pd.Series({
        'Year_min': int(df.Year.min()),
        'Year_max': int(df.Year.max()),
        'Year_mean': int(df.Year.mean()),
        'Album Count': len(df),
        'Genres': t_genres
    })

clusters_w_album_info = album_artist_cluster.groupby('Cluster').apply(get_album_cluster_info)

clusters_w_album_info = pd.merge(
    artist_clusters,
    clusters_w_album_info.reset_index(),
    on = 'Cluster'
)
clusters_w_album_info = clusters_w_album_info.sort_values(by='Year_mean', ascending=False)
clusters_w_album_info

,Cluster,Count,SampleArtists,Year_min,Year_max,Year_mean,Album Count,Genres
19,19,166,"[Wendy, Brown Eyed Girls, Jay B, Woodz, AOA, B...",1999,2021,2017,318,"[K-pop, Pop, K-pop, dance, J-pop, Hip hop, EDM..."
77,18,42,"[Digga D, Kano, Wizkid, Olamide, Headie One, P...",2003,2021,2017,71,"[Hip hop, British hip hop, Soul, R&B, hip hop,..."
35,79,120,"[Santigold, Yung Baby Tate, New Politics, Jung...",2000,2022,2017,188,"[Hip hop, Indie rock, R&B, Alternative R&B, Pop]"
20,70,158,"[Macklemore & Ryan Lewis, Mustard, Free Nation...",1993,2022,2016,421,"[Hip hop, Hip hop, trap, R&B, Alternative hip ..."
10,65,205,"[Young Ejecta, Boy Harsher, Meiko, Emilíana To...",1990,2022,2016,280,"[Indie rock, Pop, Indie pop, Synth-pop, Indie ..."
9,15,216,"[Hannah Grace, Powfu, Josh Kumra, Sinéad Harne...",1992,2022,2015,376,"[Pop, Indie rock, Indie pop, Electronic, Indie..."
86,47,18,"[Alexandra Stan, Margaret, Rita Ora & Imanbek,...",1988,2021,2014,28,"[Pop, Alternative R&B, trap, Dance, Dance, tec..."
46,72,95,"[Clarence Clarity, John Maus, Micachu and the ...",1995,2022,2014,215,"[Experimental pop, Synth-pop, Electronic, Elec..."
47,45,95,"[Troye Sivan, Charlotte Cardin, Grimes, Karen ...",1995,2021,2014,179,"[Pop, Pop, R&B, Electropop, Electronic, Electr..."
82,83,28,"[J-Ax and Fedez, Francesca Michielin, Il Volo,...",1976,2021,2013,47,"[Pop, Pop, rock, Latin pop, Blues rock, Pop ro..."
